In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
## import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import os
import json
import pickle

import re
from collections import Counter

# custom imports
from data_handling import load_data, collapse_dfs, load_large
from text_preprocessing import df_processing, party_remover

In [4]:
# pd.set_option("display.max_colwidth", -1)
pd.set_option("display.max_columns", 20)

In [7]:
read_in True
# path = '/home/franzi/Git-DSR/Twitter_sentimentanalysis/Data/large_data.pickle'
parties = ['AfD', 'SPD', 'CDU', 'CSU', 'FDP', 'LINKE', 'Gruene', 'Grüne']
dates = ['08-{:02d}'.format(i) for i in range(2,5)]

data = pd.DataFrame()
print('Loading: {}'.format(dates))
for party in parties:
    load_list = ['../Data/{}/{}_2018-{}.json'.format(party, party, day) for day in dates]
    data = load_large(load_list)

  0%|          | 0/3 [00:00<?, ?it/s]

Loading: ['08-02', '08-03', '08-04']


100%|██████████| 3/3 [00:04<00:00,  1.40s/it]


In [50]:
data.loc[:, 'user'].head()

created_at
2018-08-02 00:00:38    {'id': 21577947, 'id_str': '21577947', 'name': 'Lars', 'screen_name': 'Bildrauschen', 'location': 'Mayen-Koblenz, Rheinland-Pfalz', 'description': 'Arbeitet beruflich. Lebt privat. Quality since 1987.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 779, 'friends_count': 232, 'listed_count': 77, 'created_at': 'Sun Feb 22 16:56:24 +0000 2009', 'favourites_count': 4604, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 36256, 'lang': 'de', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'FCFFFF', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme4/bg.gif', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme4/bg.gif', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/723879233542561793/dFsPRTQw_normal.j

In [46]:
def extract_user_id(x):
    return x['id_str']

In [51]:
data.loc[:, 'user_id'] = data.user.apply(extract_user_id)

In [ ]:
data.loc[]

In [8]:
print(data.keys())

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'favorite_count', 'favorited',
       'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'metadata', 'place', 'possibly_sensitive', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'retweet_count',
       'retweeted', 'retweeted_status', 'source', 'truncated', 'user',
       'withheld_in_countries', 'withheld_scope'],
      dtype='object')


In [12]:
print(data.loc[:, 'favorited'].value_counts())

False    9153
Name: favorited, dtype: int64


In [9]:
print(len(data))

9153


In [19]:
print(data.loc[:, 'full_text'][data.loc[:, 'retweet_count'] > 5])

created_at
2018-08-02 00:02:10                                                                RT @JLessat: In der #Feinstaub #Hauptstadt #Stuttgart soll Anteil #Radverkehr auf 20 Prozent steigen, sagt OB #Kuhn (#Gruene). Wie schafft…
2018-08-02 00:20:47                                                                RT @JLessat: In der #Feinstaub #Hauptstadt #Stuttgart soll Anteil #Radverkehr auf 20 Prozent steigen, sagt OB #Kuhn (#Gruene). Wie schafft…
2018-08-02 00:22:35                                                               RT @AndreasStraub: Grüne beschuldigen Landwirte an der Dürre! Dabei sind sie Rückgrat unserer Bodenkultur, wer sie angreift bestätigt wiede…
2018-08-02 00:39:59                                                               RT @AndreasStraub: Grüne beschuldigen Landwirte an der Dürre! Dabei sind sie Rückgrat unserer Bodenkultur, wer sie angreift bestätigt wiede…
2018-08-02 00:53:07                                                          RT @UdoHemmelgarn: G